In [ ]:
%pip install wandb -Uqq

In [ ]:
%pip install nmslib -Uqq

     |████████████████████████████████| 13.5 MB 5.1 MB/s 
     |████████████████████████████████| 188 kB 47.6 MB/s 


In [ ]:
%pip install LibRecommender -Uqq

     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 24.1 MB 1.6 MB/s 


In [ ]:
%pip install faiss==1.5.3 -Uqq

     |████████████████████████████████| 4.7 MB 4.9 MB/s 


In [ ]:
%pip install ujson redis -Uqq

     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 246 kB 8.7 MB/s 


In [ ]:
import wandb
wandb.login()

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from libreco.data import random_split, split_by_ratio_chrono, DatasetPure
from libreco.algorithms import WaveNet   # pure data
from libreco.evaluation import evaluate

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from libserving.serialization import save_embed

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/WaveNet" 
if not os.path.isdir(MODEL_PATH):
    os.mkdir(MODEL_PATH)  
MODEL_NAME = 'WaveNet'

In [ ]:
data = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
data.columns = ["user", "item", "label", "time"]
data["label"] = 1 

train_data, eval_data, test_data = split_by_ratio_chrono(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
eval_data.build_negative_samples(data_info) 
test_data.build_negative_samples(data_info)
print(data_info)   # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

random neg item sampling elapsed: 0.168s
random neg item sampling elapsed: 0.018s
random neg item sampling elapsed: 0.031s
n_users: 610, n_items: 8237, data sparsity: 1.6056 %


In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [ ]:
# from IPython.lib.display import exists
# with wandb.init() as runwb:
#     # Overwrite the random run names chosen by wandb
#     runwb.config.update(config)
#     sc = runwb.config
#     name_str = f'{MODEL_NAME}' + \
#       f'_n_epoch_{sc.n_epochs}_' + \
#       f'emb_size_{sc.embed_size}_lr_{sc.lr}_' + \
#       f'batch_size_{sc.batch_size}'
#     runwb.config.update({"name": name_str})
#     current_dir = os.path.join(MODEL_PATH, name_str)
#     if not os.path.isdir(current_dir):
#         os.mkdir(current_dir)  
from types import SimpleNamespace  

cfg = {
  "embed_size": 16,
  "n_epochs": 3,
  "lr": 0.01, 
  "batch_size": 256,
  "lr_decay":True,
  "loss_type":"cross_entropy",
  "num_neg":1,
  "dropout_rate": None,
  "k":10,
  "use_bn":False, 
  "n_filters":2, 
  "n_blocks":1, 
  "n_layers_per_block":4, 
  "recent_num":10, 
  "random_num":None,
}
cfg = SimpleNamespace(**cfg)

metrics = ["loss", "roc_auc", "precision", "recall", "map", "ndcg"]    
# Rest of code.
model = WaveNet(task="ranking", data_info=data_info, 
                embed_size=cfg.embed_size, n_epochs=cfg.n_epochs, 
                lr=cfg.lr, batch_size=cfg.batch_size)
model.fit(train_data, verbose=2, eval_data=eval_data, metrics=metrics)
evaluate(model=model, data=test_data, metrics=metrics)
# specify model saving directory
name_str = f'{MODEL_NAME}' + \
  f'_n_epoch_{cfg.n_epochs}_' + \
  f'emb_size_{cfg.embed_size}_' + \
  f'lr_{cfg.lr}_' + \
  f'batch_size_{cfg.batch_size}'
current_dir = os.path.join(MODEL_PATH, name_str)
if not os.path.isdir(current_dir):
    os.mkdir(current_dir) 
save_embed(current_dir, model)        
model.save(path=current_dir, model_name="Caser", manual=True, inference_only=True)

total params: 416,045 | embedding params: 413,485 | network params: 2,560
Training start time: 2022-11-06 12:57:09


train: 100%|██████████| 631/631 [00:16<00:00, 37.23it/s]


Epoch 1 elapsed: 17.016s
	 train_loss: 0.4889


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 2262.60it/s]


	 eval log_loss: 0.5272
	 eval roc_auc: 0.8219
	 eval precision@10: 0.0289
	 eval recall@10: 0.0297
	 eval map@10: 0.0686
	 eval ndcg@10: 0.1063


...

train: 100%|██████████| 631/631 [00:17<00:00, 36.13it/s]


Epoch 3 elapsed: 17.517s
	 train_loss: 0.1683


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 1727.13it/s]


	 eval log_loss: 0.7341
	 eval roc_auc: 0.8105
	 eval precision@10: 0.0252
	 eval recall@10: 0.0235
	 eval map@10: 0.0679
	 eval ndcg@10: 0.1002


eval_rec: 100%|██████████| 608/608 [00:00<00:00, 2183.39it/s]


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/WaveNet" 
MODEL_NAME = 'WaveNet'

sweep_config = {
    "name": "WaveNet-sweep",
    "metric": {"name": "loss", "goal": "minimize"},    
    "method": "random",  
    'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },  
    "parameters": {        
          "embed_size": {
              "values": [16, 32]
          },
          'n_epochs': {
              "values": [5,10]
          }, 
          "lr": {
              "values": [0.1, 0.01, 0.001, 0.0001]
          },
                    
          "batch_size": {
              "values": [64, 128, 256]
          },
          }
}
metrics = ["loss", "roc_auc", "precision", "recall", "map", "ndcg"]    
sweep_id = wandb.sweep(sweep_config, 
                       project="recsys-app", 
                       entity="iloncka-ds")

os.environ["WANDB_RUN_GROUP"] = "experiment-WaveNet"

def train_with_wandb():  
      with wandb.init(group=MODEL_NAME,
                      notes=f"{MODEL_NAME} hyperparameters search",
                      tags=["embed", "sequence", "hyperparameters"]) as runwb:
        # Overwrite the random run names chosen by wandb
        sc = runwb.config
        name_str = f'{MODEL_NAME}_met_{sweep_config["method"]}' + \
          f'_n_epoch_{sc.n_epochs}_' + \
          f'emb_size_{sc.embed_size}_lr_{sc.lr}_' + \
          f'batch_size_{sc.batch_size}'
        runwb.config.update({"name": name_str})
        current_dir = os.path.join(MODEL_PATH, name_str)
        if not os.path.isdir(current_dir):
            os.mkdir(current_dir)  
        # Rest of code.
        model = WaveNet(task="ranking", data_info=data_info, 
                        embed_size=sc.embed_size, n_epochs=sc.n_epochs, 
                        lr=sc.lr, reg=None, batch_size=sc.batch_size, lr_decay=True,)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        model.fit(runwb, train_data, verbose=2, eval_data=eval_data, metrics=metrics)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        evaluate(runwb=runwb, model=model, data=test_data,
                                    metrics=metrics)
        # specify model saving directory
        save_embed(current_dir, model)        
        model.save(path=current_dir, model_name="WaveNet", manual=True, inference_only=True)
wandb.agent(sweep_id, function=train_with_wandb, entity="iloncka-ds", project="recsys-app", count=20)

Create sweep with ID: r5ls7bmj
Sweep URL: https://wandb.ai/iloncka-ds/recsys-app/sweeps/r5ls7bmj


wandb: Agent Starting Run: h9hhcnnr with config:
wandb: 	batch_size: 128
wandb: 	embed_size: 32
wandb: 	lr: 0.1
wandb: 	n_epochs: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


total params: 821,981 | embedding params: 818,653 | network params: 3,328
Training start time: 2022-11-06 14:02:56
With lr_decay, epoch 1 learning rate: 0.10000000149011612


train: 100%|██████████| 1261/1261 [00:24<00:00, 50.69it/s]


Epoch 1 elapsed: 24.951s
	 train_loss: 3.4837


...

Epoch 5 elapsed: 22.597s
	 train_loss: 0.0705


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 1570.59it/s]


	 eval log_loss: 0.9689
	 eval roc_auc: 0.8091
	 eval precision@10: 0.0238
	 eval recall@10: 0.0229
	 eval map@10: 0.0692
	 eval ndcg@10: 0.0996


eval_rec: 100%|██████████| 608/608 [00:00<00:00, 1647.42it/s]


epoch,▁▃▅▆█
log_loss,▁▂▄▆█
loss,▁
map,▁
map@10,█▂▁▁▃
ndcg,▁
ndcg@10,█▄▂▁▂
precision,▁
precision@10,█▅▂▁▁
recall,▁
recall@10,█▅▃▁▄


https://wandb.ai/iloncka-ds/recsys-app/groups/WaveNet/workspace?workspace=user-iloncka-ds